# 행렬 분해 (1)

## LU Decomposition
- 선형 대수 강좌에서는 LU decomposition 에서 interchange를 사용하지 않았다. Decomposition을 진행하는 경우 pivot position이 0인 경우 interchange가 필요하다.
$\to$ Row interchange $\Longrightarrow$ pivoting
- 기본 형태: A = LU
- SciPy: A = PLU
    - P: pivoting 결과를 담고 있는 permutation 행렬(row interchange의 정보가 들어있다.), Identity matrix

- **linalg.lu**
- P, L, U = linalg.lu(A)

In [1]:
import numpy as np
from scipy import linalg

In [5]:
A = np.array([[2,4,-1,5,-2],[-4,-5,3,-8,1],[2,-5,-4,1,8],[-6,0,7,-3,1]])
P, L, U = linalg.lu(A)
print("P: ", P, "L: ", L, "U: ", U)

P:  [[0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]] L:  [[ 1.          0.          0.          0.        ]
 [ 0.66666667  1.          0.          0.        ]
 [-0.33333333  1.          1.          0.        ]
 [-0.33333333 -0.8         0.13333333  1.        ]] U:  [[-6.00000000e+00  0.00000000e+00  7.00000000e+00 -3.00000000e+00
   1.00000000e+00]
 [ 0.00000000e+00 -5.00000000e+00 -1.66666667e+00 -6.00000000e+00
   3.33333333e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.22044605e-16  6.00000000e+00
   8.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.60000000e+00
  -2.46666667e+00]]


## LU Decomposition for Solver
- A = PLU
- linalg.lu_factor
- lu, piv = linalg.lu_factor(A)
    - lu: L과 U를 한 행렬에 저장
    - piv: 1D array(vector): row interchange 정보 저장
        - piv: [3(0),1(1),2(2),3(3)]: 
            - (0)$\leftrightarrow$3
            - (1)$\leftrightarrow$1
            - (2)$\leftrightarrow$2
            - (3)$\leftrightarrow$3
            

In [15]:
A = np.array([[7,5,6,6],[1,2,2,8],[5,4,4,8],[9,5,8,7]])
lu, piv = linalg.lu_factor(A)
# L, U 구축
L = np.tril(lu,k=-1) + np.eye(lu.shape[0], lu.shape[1])
U = np.triu(lu)
print("LU: ", lu)
print()
print("piv: ", piv)
print("L: ", L)
print()
print("U: ", U)

# P 구축
perm = np.copy(piv)
P = np.identity(4)[perm,:]
A = P@L@U
A = (L@U)[perm,:]
print("A: ", A)

LU:  [[ 9.          5.          8.          7.        ]
 [ 0.11111111  1.44444444  1.11111111  7.22222222]
 [ 0.55555556  0.84615385 -1.38461538 -2.        ]
 [ 0.77777778  0.76923077  0.77777778 -3.44444444]]

piv:  [3 1 2 3]
L:  [[1.         0.         0.         0.        ]
 [0.11111111 1.         0.         0.        ]
 [0.55555556 0.84615385 1.         0.        ]
 [0.77777778 0.76923077 0.77777778 1.        ]]

U:  [[ 9.          5.          8.          7.        ]
 [ 0.          1.44444444  1.11111111  7.22222222]
 [ 0.          0.         -1.38461538 -2.        ]
 [ 0.          0.          0.         -3.44444444]]
A:  [[7. 5. 6. 6.]
 [1. 2. 2. 8.]
 [5. 4. 4. 8.]
 [7. 5. 6. 6.]]


## LU Decomposition Solver
- linalg.lu_factor의 결과를 활용하여 $Ax = b$를 푸는 solver
- Ax = b의 형태에서 b를 바꿔가면서 해를 구할때 유용
- linalg.lu_solve
- x = linalg.lu_solve((lu, piv), b)
    - lu, piv = linalg.lu_factor(A)

In [34]:
import timeit

A = np.array([[7,5,6,6],[1,2,2,8],[5,4,4,8],[9,5,8,7]])
b = np.ones((4,))

lu, piv = linalg.lu_factor(A)

start = timeit.default_timer()
x = linalg.lu_solve((lu, piv), b)
end = timeit.default_timer()
print(x)
print()
print("computing time by using LU Decomposition: ", end-start)


[-0.16129032  0.19354839  0.12903226  0.06451613]

computing time by using LU Decomposition:  0.00036393100026543834


In [21]:
start = timeit.default_timer()
x = linalg.solve(A, b)
end = timeit.default_timer()
print(x)
print()
print("computing time by using general solve method: ", end-start)

[-0.16129032  0.19354839  0.12903226  0.06451613]

computing time by using general solve method:  0.0015140679897740483


#### linalg.solve method는 LU Decomposition을 해서 풀지만 실제로 Decomposition 된것의 이점은 활용하지 못함

### 1000 x 1000 matrix 에서 .solve V.S .lu_solve V.S .solve_banded 연산 속도 비교

In [35]:
diag_line = 5 * np.ones((1000,))
first_band_line = np.ones((999,))
minus_band_line = np.ones((999,))

A = np.diag(diag_line) + np.diag(first_band_line, k=1) + np.diag(minus_band_line, k=-1)
b = np.ones((1000,))
print("A: ", A)


A:  [[5. 1. 0. ... 0. 0. 0.]
 [1. 5. 1. ... 0. 0. 0.]
 [0. 1. 5. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 5. 1. 0.]
 [0. 0. 0. ... 1. 5. 1.]
 [0. 0. 0. ... 0. 1. 5.]]


In [36]:
# .solve
start = timeit.default_timer()
x = linalg.solve(A, b)
end = timeit.default_timer()

print("computing time: ", end - start)


computing time:  0.014596219989471138


In [37]:
# .lu_solve
lu, piv = linalg.lu_factor(A)
start = timeit.default_timer()
x = linalg.lu_solve((lu, piv), b)
end = timeit.default_timer()

print("computing time: ", end - start)

computing time:  0.0007312569650821388


In [38]:
# .solve_banded
zr = np.zeros((1,))
ones = np.ones((999,))

first_band_row = np.hstack((zr, ones))
mid_band_row = 5 * np.ones((1000,))
last_band_row = np.hstack((ones, zr))

band_A = np.vstack((first_band_row, mid_band_row))
band_A = np.vstack((band_A, last_band_row))

start = timeit.default_timer()
x = linalg.solve_banded((1,1), band_A, b)
end = timeit.default_timer()

print("computing time: ", end-start)

computing time:  0.00027194502763450146


#### .solve_banded > .lu_solve > .solve

## Diagonal Pivoting Method
- Symmetric & complex symmetric
    - $A = LD{L}^T / A = UD{U}^T$
        - Lapack: sytrf
- Hermitian
    - $A = LD{L}^* / A = UD{U}^*$
        - Lapack: hetrf
- D: block digoanal 최대 2x2 block
- L: lower(or upper) triangular matrix(diagonal=1)와 permutation matrix의 multiplication

- **linalg.ldl**
- L, D, perm = linalg.ldl(A, lower=True, hermitian=True)
- A = L @ D @ L.T
- A[perm,:][:,perm] = L[perm,:] @ D @ L[perm,:].T
    - $PA{P}^T = (PL)D{(PL)}^T$
        : L도 interchange가 되었으니 A도 interchange를 한 것!

# 행렬 분해 (2)

## Cholesky Decomposition
- pivoting 없이도 수치적으로 매우 안정적
- LU 보다 대략 두배 빠름
- positive definite(symmetric / Hermitian)
- $A = {R}^TR / A = {R}^*R$
- $A = {L}{L}^T / A = {L}{L}^*$

- **linalg.cholesky**
    - Lapack: potrf
- R = linalg.cholesky(A, lower=False)

## Cholesky Decomposition Solver
- Ax = b
- **linalg.cho_solve**
- x = linalg.cho_solve((R, False), b)
    False: ${R}^TR$의 형태를 넣겠다. if True: ${L}{L}^T / {L}{L}^*$

In [48]:
A1 = np.array([[1, -2j],[2j, 5]], dtype=np.complex128)
b1 = np.ones((2,))

R1 = linalg.cholesky(A1, lower=False)
x1 = linalg.cho_solve((R1, False),b1)

print("verification: ", np.allclose(A1@x1, b1))

verification:  True


In [50]:
A2 = np.array([[1, -1, 0],[-1, 2, -1], [0, -1, 3]], dtype=np.complex128)
b2 = np.ones((3,))

R2 = linalg.cholesky(A2, lower=False)
x2 = linalg.cho_solve((R2, False),b2)

print("verification: ", np.allclose(A2@x2, b2))

verification:  True


#### LU decomposition후 해를 구하는 것이 더 빠르다.
- decomposition 자체 속도는 cholesky가 더 빠르다. decomposition 후 해를 구하는 경우: LU Decomposition solver 승리
- L 내부의 Lapack에서 자체적으로 diagonald entry 1, 즉 쓸데 없는 연산을 안함
    - 1) A 고정 b가 많이 변하는 상황: LU
    - 2) A를 많이 변화시켜야하는 상황 or 단일 문제: Cholesky

## 밴드 행렬의 Cholesky Decomposition
- **linalg.cholesky_banded**
- R_band = linalg.cholesky_banded(band_A_h, lower=False)
    - band_A_h: read_banded_h라는 custom 함수의 결과

In [59]:
A = np.array([[9,-1,1j,0,0],[-1,8,-2,2,0],[-1j,-2,7,3,3j],[0,2,3,6,4],[0,0,-3j,4,9]])
b = np.ones((5,))
zr1 = np.zeros((1,))
zr2 = np.zeros((2,))

first_band_entries = np.array([A[0,2], A[1,3], A[2,4]])
second_band_entries = np.array([A[0,1], A[1,2], A[2,3], A[3,4]])
third_band_entries = np.array([9,8,7,6,9])

first_band_row = np.hstack((zr2, first_band_entries))
second_band_row = np.hstack((zr1, second_band_entries))

upper_band_A = np.vstack((first_band_row, second_band_row))
upper_band_A = np.vstack((upper_band_A, third_band_entries))

R_band = linalg.cholesky_banded(upper_band_A, lower=False)
R_band

array([[ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.33333333j,  0.71206899+0.j        ,
         0.        +1.18768515j],
       [ 0.        +0.j        , -0.33333333+0.j        ,
        -0.71206899+0.03955939j,  1.38842067+0.01115197j,
         2.11145768-0.87334379j],
       [ 3.        +0.j        ,  2.80871659+0.j        ,
         2.52592195+0.j        ,  1.88815291+0.j        ,
         1.53896753+0.j        ]])

## 밴드 행렬의 Cholesky Decomposition Solver
- **linalg.cho_solve_banded**
- x = linalg.cho_solve_banded((R_band, False), b)

In [61]:
x = linalg.cho_solve_banded((R_band, False), b)
x

array([ 0.1524183 -0.06901961j,  0.44026144+0.01385621j,
        0.63503268-0.06849673j, -0.54980392-0.15843137j,
        0.37830065+0.2820915j ])

## 행렬 분해와 Ax = b 전략
- Real symmetric / complex Hermitian?
    - Y $\to$ Positive definite?
        - Y $\to$ Cholesky
        - N $\to$ $LD{L}^T$ or $LD{L}^*$
        
    - N $\to$ Complex symmetric?
        - Y $\to$ $LD{L}^T$
        - N $\to$ $LU$
- 현재 $LD{L}^T$ decomposition solver가 없으니 LU 사용

In [67]:
# 연습 문제 
## Practice 1.
first_band_entries = 1j * np.ones((9998,))
second_band_entries = np.ones((9999,))
mid_band_entries = 5 * np.ones((10000,))
last_band_entries = -first_band_entries

A = np.diag(mid_band_entries) + np.diag(second_band_entries, k=1) + np.diag(first_band_entries, k=2) + np.diag(second_band_entries, k=-1) \
+ np.diag(last_band_entries, k=-2)
b = np.ones((10000,))

In [69]:
# LU decomposition and solution
start = timeit.default_timer()
lu, piv = linalg.lu_factor(A)
end = timeit.default_timer()
print("decomposition time: ", end-start)

start = timeit.default_timer()
x = linalg.lu_solve((lu, piv), b)
end = timeit.default_timer()
print("solving solution time: ", end-start)

decomposition time:  13.817694955971092
solving solution time:  0.0647548430133611


In [70]:
# Cholesky decomposition and solution
start = timeit.default_timer()
R = linalg.cholesky(A)
end = timeit.default_timer()
print("decomposition time: ", end-start)

start = timeit.default_timer()
x = linalg.cho_solve((R, False), b)
end = timeit.default_timer()
print("decomposition time: ", end-start)


decomposition time:  7.712093210022431
decomposition time:  0.2168013490154408


In [77]:
# Cholesky decomposition / band matrix solution
first_band_entries = 1j * np.ones((9998,))
second_band_entries = np.ones((9999,))
mid_band_entries = 5 * np.ones((10000,))
last_band_entries = -first_band_entries
zr1 = np.zeros((1,))
zr2 = np.zeros((2,))

first_row = np.hstack((zr2, first_band_entries))
second_row = np.hstack((zr1, second_band_entries))
mid_row = mid_band_entries
forth_row = np.hstack((second_band_entries, zr1))
fifth_row = np.hstack((last_band_entries, zr2))

band_A = np.vstack((first_row, second_row))
band_A = np.vstack((band_A, mid_band_entries))
# band_A = np.vstack((band_A, forth_row))
# band_A = np.vstack((band_A, fifth_row))

band_A

array([[0.+0.j, 0.+0.j, 0.+1.j, ..., 0.+1.j, 0.+1.j, 0.+1.j],
       [0.+0.j, 1.+0.j, 1.+0.j, ..., 1.+0.j, 1.+0.j, 1.+0.j],
       [5.+0.j, 5.+0.j, 5.+0.j, ..., 5.+0.j, 5.+0.j, 5.+0.j]])

In [82]:
start = timeit.default_timer()
R_band = linalg.cholesky_banded(band_A, lower=False)
end = timeit.default_timer()
print("decomposition time: ", end-start)

start = timeit.default_timer()
x = linalg.cho_solve_banded((R_band, False), b)
end = timeit.default_timer()
print("decomposition time: ", end-start)


decomposition time:  0.0015332840266637504
decomposition time:  0.0010570089798420668


# Low-Level Lapack 함수의 활용
현재 SciPy에서 $LD{L}^T$ low-level Lapack 함수를 제공 안함,  Cython을 사용하면 가능
- **linalg.get_lapack_funcs**

## 밴드 행렬  LU Decomposition
- gbtrf: 밴드 행렬의 LU Decomposition
- gbtrf = linalg.get_lapack_funcs("gbtrf", dtype=np.float64)
- LU_band, piv, info = gbtrf(A_band_LU, lbw, ubw)
    - LU_band: L, U를 하나의 band형태로 저장
    - piv: 1D array (vector): row interchange 정보를 저장
    - info: 함수가 제대로 돌았나, i=0: 정상, i>0: singular, i<0: 잘못된 입력
    - A_band_LU: 우리가 배운 A_band와는 다름
        - 기존 A_band row size: lbw + ubw + 1
        - A_band_LU row size: (ld) = lbw * 2 + ubw + 1 (lower band width * 2이다.)
    - dummy_array = np.zeros((lbw, A_band.shape[1]))
    - A_band_LU = np.vstack((dummy_array, A_band))

In [8]:
first_band = 2 * np.ones((4,))
diag_band = np.ones((5,))
first_low_band = np.ones((4,))
second_low_band = 2 * np.ones((3,))

zr1 = np.zeros((1,))
zr2 = np.zeros((2,))


A = np.diag(diag_band) + np.diag(first_band, k=1) + np.diag(first_low_band, k=-1) + np.diag(second_low_band, k=-2)

first_band_row = np.hstack((zr1, first_band))
lower_first_row = np.hstack((first_low_band, zr1))
lower_second_row = np.hstack((second_low_band, zr2))

band_A = np.vstack((first_band_row, diag_band))
band_A = np.vstack((band_A, lower_first_row))
band_A = np.vstack((band_A, lower_second_row))

print("Matrix A: ", A)
print()
print("band_A: ", band_A)

Matrix A:  [[1. 2. 0. 0. 0.]
 [1. 1. 2. 0. 0.]
 [2. 1. 1. 2. 0.]
 [0. 2. 1. 1. 2.]
 [0. 0. 2. 1. 1.]]

band_A:  [[0. 2. 2. 2. 2.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 0.]
 [2. 2. 2. 0. 0.]]


In [11]:
dummy_array = np.zeros((2, band_A.shape[1]))
A_band_LU = np.vstack((dummy_array, band_A))

print("dummy_array: ", dummy_array)
print()
print("A_band_LU: ", A_band_LU)

dummy_array:  [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]

A_band_LU:  [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 2. 2. 2. 2.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 0.]
 [2. 2. 2. 0. 0.]]


## 밴드 행렬 LU Decomposition Solver
- gbtrs: 밴드 행렬의 LU dcomposition solver
- gbtrs = linalg.get_lapack_funcs("gbtrs", dtype=np.float64)
- x, info = gbtrs(LU_band, lbw, ubw, b, piv)
    - info: i = 0: 정상, i < 0: 잘못된 입력
    - LU_band, piv: gbtrf의 결과

In [29]:
gbtrf = linalg.get_lapack_funcs("gbtrf", dtype=np.float64)
gbtrs = linalg.get_lapack_funcs("gbtrs", dtype=np.float64)

dummy_array = np.zeros((2, band_A.shape[1]))  #(2, 5)
A_band_LU = np.vstack((dummy_array, band_A))
b = np.ones((5,))

LU_band, piv, info = gbtrf(A_band_LU, 2, 1)
x, info = gbtrs(LU_band, 2, 1, b, piv)

print(np.allclose(A@x, b))

True


## lu_factor / lu_solve V.S gbtrf / gbrts

In [41]:
diag_entries = 5 * np.ones((1000,))
first_upper_band = np.ones((999,))
first_lower_band = np.ones((999,))
zr1 = np.zeros((1,))
zr2 = np.zeros((2,))

first_band_row = np.hstack((zr1, first_upper_band))
third_band_row = np.hstack((first_lower_band, zr1))

A = np.diag(diag_entries) + np.diag(first_upper_band, k=1) + np.diag(first_lower_band, k=-1)
A_band = np.vstack((first_band_row, diag_entries))
A_band = np.vstack((A_band, third_band_row))
b = np.ones((1000,))

print("A: ", A)
print()
print("A_band: ", A_band)

A:  [[5. 1. 0. ... 0. 0. 0.]
 [1. 5. 1. ... 0. 0. 0.]
 [0. 1. 5. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 5. 1. 0.]
 [0. 0. 0. ... 1. 5. 1.]
 [0. 0. 0. ... 0. 1. 5.]]

A_band:  [[0. 1. 1. ... 1. 1. 1.]
 [5. 5. 5. ... 5. 5. 5.]
 [1. 1. 1. ... 1. 1. 0.]]


In [42]:
start1 = timeit.default_timer()
lu, piv = linalg.lu_factor(A)
end1 = timeit.default_timer()
print("LU decomposition time: ", end1-start1)

start2  = timeit.default_timer()
x = linalg.lu_solve((lu, piv), b)
end2 = timeit.default_timer()
print("x time: ", end2-start2)
print("total time: ", (end1-start1) + (end2-start2))

LU decomposition time:  0.01643508000051952
x time:  0.0009049760001289542
total time:  0.017340056000648474


In [43]:
# Construct A_band_LU
gbtrf = linalg.get_lapack_funcs("gbtrf", dtype=np.float64)
gbtrs = linalg.get_lapack_funcs("gbtrs", dtype=np.float64)

dummy_array = np.zeros((1, A_band.shape[1]))
A_band_LU = np.vstack((dummy_array, A_band))

A_band_LU

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 1., 1., 1.],
       [5., 5., 5., ..., 5., 5., 5.],
       [1., 1., 1., ..., 1., 1., 0.]])

In [44]:
start3 = timeit.default_timer()
LU_band, piv, info = gbtrf(A_band, 1, 1)
end3 = timeit.default_timer()
print("gbtrf time: ", end3 - start3)

start4 = timeit.default_timer()
x, info = gbtrs(LU_band, 1,1, b, piv)
end4 = timeit.default_timer()
print("gbtrs time: ", end4 - start4)
print('total time: ', (end4 - start4) + (end3 - start3))


gbtrf time:  0.00024131099962687586
gbtrs time:  0.000177590000021155
total time:  0.00041890099964803085


#### Lapack  함수를 이용한 solver가 더 시간이 적게 걸렸다.

# 행렬 분해 (3)
## QR Decomposition
- A = QR (Q: orthogonal / unitary, R: upper triangular matrix)
- **linalg.qr**
    - Lapck: geqrf, orgqr / ungqr
- Q, R = linalg.qr(A, mode="economic")
    - mode = "economic"으로 하면 불필요한 0들을 제외하고 연산하여 더 효울적임, 기본적으로 "economic"사용을 권장

In [51]:
A = np.tri(4,3)
Q, R = linalg.qr(A, mode="economic")
print(np.allclose(A, Q@R))

True


## One-step Approach QR Algorithm 흉내내보기
- $A = {Q}_1{R}_1 \to {A}_1 = {R}_1{Q}_1$, ${A}_1$ is similar to A
- ${A}_1 = {Q}_2{R}_2 \to {A}_2 = {R}_2{Q}_2$, ${A}_2$ is similar to A
- $\vdots$
- ${A}_{k-1} = {Q}_k{R}_k \to {A}_k = {R}_k{Q}_k$, ${A}_k$ is similar to A
- ${A}_k$ is upper triangular matrix
$\Longrightarrow$ ${A}_k$의 대각선 entries: eigenvalues! 

In [62]:
A = np.array([[1,3,3], [-3,-5,-3], [3,3,1]])
eigvals = linalg.eig(A, right=False)
Ak = A
for k in range(0,100):
    Q, R = linalg.qr(Ak)
    Ak = R@Q
    print(k)
    print(Ak)

0
[[-0.89473684 -0.94257159 -8.2171486 ]
 [ 0.33663271 -2.28708134 -2.5027171 ]
 [-0.29346959  0.25027171  0.18181818]]
1
[[-2.78947368  0.27078724  5.90167107]
 [-0.75820427 -1.73993808  5.66791813]
 [-0.47213369  0.16194052  1.52941176]]
2
[[-1.65030675 -0.23261249 -7.72147344]
 [ 0.16650157 -2.1107552  -3.67647206]
 [-0.1250441   0.0831781   0.76106195]]
3
[[-2.19290466  0.10164088  7.09228963]
 [-0.1241721  -1.9345741   4.56528391]
 [-0.08506494  0.04482046  1.12747875]]
4
[[-1.90775585 -0.05542518 -7.45732128]
 [ 0.05060771 -2.0304078  -4.09129432]
 [-0.03634394  0.02183736  0.93816365]]
5
[[-2.04723517  0.02669637  7.28948697]
 [-0.02799846 -1.98417583  4.32081471]
 [-0.01964325  0.01110197  1.031411  ]]
6
[[-1.97665475e+00 -1.36179681e-02 -7.37679133e+00]
 [ 1.33049207e-02 -2.00776115e+00 -4.20417936e+00]
 [-9.44474854e-03  5.50939816e-03  9.84415898e-01]]
7
[[-2.01174155e+00  6.74357474e-03  7.33401977e+00]
 [-6.82337974e-03 -1.99608110e+00  4.26202779e+00]
 [-4.81543461e-03  2

## Hessenberg Reduction